In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_row',500)
pd.options.display.max_columns = 150

In [2]:
#raw test data
df=pd.read_csv('C:/Users/hp/Desktop/DSP/New folder (2)/Python/Credit risk/Test_data.csv')

In [3]:
#pre processed train data
df_train=pd.read_csv('C:/Users/hp/Desktop/DSP/New folder (2)/Python/Credit risk/Preprocessed_Train_data.csv')

In [4]:
df.shape

(256991, 73)

In [4]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,default_ind
0,30305029,32848196,12000,12000,12000.0,36 months,9.17,382.55,B,B1,Owner,10+ years,MORTGAGE,60000.0,Not Verified,Oct-14,n,NaN,small_business,Business,795xx,TX,4.38,0,May-99,0,68.0,NaN,8,0,13425,24.1,15,f,7395.97,7395.97,5732.14000,5732.14,4604.03,1128.11,0.0,0.0,0.0,Jan-16,382.55,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0.0,13425.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55700.0,NaN,NaN,NaN,0
1,1510647,1772435,16000,16000,16000.0,36 months,15.80,560.94,C,C3,Final Mile Logistics,2 years,RENT,200000.0,Source Verified,Sep-12,n,Borrower added on 09/04/12 > I am using this...,debt_consolidation,Consolidating Debit,302xx,GA,10.18,0,Jun-01,2,43.0,NaN,19,0,11488,55.5,32,f,0.00,0.00,19032.52004,19032.52,16000.00,3032.52,0.0,0.0,0.0,Mar-14,9502.32,NaN,Jan-16,0.0,70.0,1,INDIVIDUAL,NaN,NaN,NaN,0,115.0,89648.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20700.0,NaN,NaN,NaN,0
2,7437189,9099270,24575,24575,24575.0,60 months,17.10,612.08,C,C5,JPMorgan Chase,2 years,MORTGAGE,117200.0,Source Verified,Sep-13,n,Borrower added on 09/19/13 > 100% of this lo...,debt_consolidation,Debt consolidation,335xx,FL,26.45,2,Dec-96,0,6.0,NaN,12,0,28572,89.3,31,w,0.00,0.00,31825.17000,31825.17,24575.00,7250.17,0.0,0.0,0.0,Sep-15,18359.41,NaN,Jan-16,0.0,6.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0.0,404433.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32000.0,NaN,NaN,NaN,0
3,25217448,27660173,2000,2000,2000.0,36 months,14.99,69.33,C,C5,Business Analyst,4 years,RENT,43400.0,Not Verified,Aug-14,n,NaN,moving,Moving and relocation,152xx,PA,12.58,0,Feb-05,2,NaN,NaN,12,0,4198,29.6,18,f,0.00,0.00,2128.91000,2128.91,2000.00,128.91,0.0,0.0,0.0,Jan-15,1851.59,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0.0,23990.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14200.0,NaN,NaN,NaN,0
4,33441379,36084663,9600,9600,9600.0,36 months,12.39,320.65,C,C1,warehouse,6 years,RENT,45000.0,Source Verified,Nov-14,n,NaN,debt_consolidation,Debt consolidation,450xx,OH,23.23,0,Jun-01,1,65.0,80.0,11,1,11544,49.5,23,w,6534.77,6534.77,4161.84000,4161.84,3065.23,1096.61,0.0,0.0,0.0,Dec-15,320.65,Feb-16,Jan-16,0.0,65.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0.0,26097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23300.0,NaN,NaN,NaN,0


In [5]:
#Dealing with date variables
year_feature=[feature for feature in ['issue_d','earliest_cr_line','last_pymnt_d','next_pymnt_d','last_credit_pull_d']]
for feature in year_feature:
    df[feature]=pd.to_datetime(df[feature].str.upper(), format='%b-%y', yearfirst=False)

In [6]:
print(df['default_ind'].value_counts(),df['default_ind'].value_counts()/len(df['default_ind'])*100)
#This is an imbalanced dataset

0    256680
1       311
Name: default_ind, dtype: int64 0    99.878984
1     0.121016
Name: default_ind, dtype: float64


In [7]:
#Length of credit history in years
 
df['cred_hist_len_years']=df['issue_d'].dt.year-df['earliest_cr_line'].dt.year

df['cred_hist_len_years'].head()

0    19
1    10
2    19
3    26
4    22
Name: cred_hist_len_years, dtype: int64

In [8]:
df['term']=df['term'].apply(lambda x: x.rstrip('months'))
df['term']=df['term'].astype('int64')

In [9]:
df['emp_length']=df['emp_length'].map({'10+ years':'10','< 1 year':'0.5','1 year':'1','3 years':'3','8 years':'8','9 years':'9',
                      '4 years':'4','5 years':'5','6 years':'6','2 years':'2','7 years':'7'})
df['emp_length']=df['emp_length'].astype('float64')
df['emp_length'].head()

0     1.0
1     7.0
2    10.0
3     7.0
4     2.0
Name: emp_length, dtype: float64

In [10]:
df['pub_rec']=np.where(df['pub_rec']>0.0,1,0)
df['recoveries']=np.where(df['recoveries']>0.0,1,0)
df['delinq_2yrs']=np.where(df['delinq_2yrs']>0.0,1,0)

In [13]:
df.loc[(df['dti']==9999.0), ['installment','annual_inc','dti']]

,installment,annual_inc,dti
10305,712.96,0.0,9999.0
56704,123.94,0.0,9999.0


In [14]:
#dti is debt to income ratio
#dti=9999.0 for missing values in annual income hence changing it to zero
df['dti']=np.where(df['dti']==9999.0,0.0,df['dti'])

In [11]:
df_train.shape

(598978, 32)

In [15]:
final_cols=df_train.columns

In [16]:
df=df[final_cols]

In [17]:
df.shape

(256991, 32)

In [18]:
numerical_features=[feature for feature in df.columns if df[feature].dtypes!='O']
print('Numerical features are:', len(numerical_features))


Numerical features are: 26


In [19]:
discrete_feature=[feature for feature in numerical_features if len(df[feature].unique())<25 and feature not in year_feature]
print('discrete feature are:{}'.format(len(discrete_feature)))

discrete feature are:7


In [20]:
continuous_feature=[feature for feature in numerical_features if feature not in discrete_feature]
print('Continuous feature are: {}'.format(len(continuous_feature)))

Continuous feature are: 19


In [21]:
## Let us capture all the nan values
## First lets handle Categorical features which are missing
features_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes=='O']
for feature in features_nan:
    print('{}:{}% missing values'.format(feature,np.round(df[feature].isnull().mean()*100,4)))

In [22]:
## Now lets check for numerical variables the contains missing values
numerical_with_nan=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes!='O']
## We will print the numerical nan variables and percentage of missing values
for feature in numerical_with_nan:
    print('{}:{}% missing values'.format(feature,np.round(df[feature].isnull().mean()*100),4))

emp_length:6.0% missing values
mths_since_last_delinq:49.0% missing values
revol_util:0.0% missing values


In [23]:
for feature in numerical_with_nan:
    ## We will replace nans with zero since median/median imputation is not the right choice for this data
    

    df[feature].fillna(df[feature]==0,inplace=True)
    df[feature]=df[feature].astype('float64')
    
df[numerical_with_nan].isnull().sum()

emp_length                0
mths_since_last_delinq    0
revol_util                0
dtype: int64

In [24]:
#Handling skewness

for feature in continuous_feature:
    if ((df[feature].skew()>1.0) | (df[feature].skew()<-1.0)):
        df[feature]=np.log(df[feature]+1)
        print(feature,df[feature].skew())

annual_inc 0.17638710504621244
dti -1.1273106417430294
mths_since_last_delinq 0.21680411901633084
open_acc -0.10313138224034601
revol_bal -2.1252176720948683
total_pymnt -2.472769547304891
total_rec_prncp -2.110644807479879
total_rec_int -1.871523851930498
last_pymnt_amnt -1.9955648053089152
tot_coll_amt 2.1282347538694775
tot_cur_bal -0.506804209952807
total_rev_hi_lim -0.7373639032149354


In [ ]:
## Handling Rare Categorical Feature

We will remove categorical variables that are present less than 1% of the observations


In [25]:
categorical_features=[feature for feature in df.columns if df[feature].dtypes=='O']
categorical_features

['grade',
 'sub_grade',
 'home_ownership',
 'verification_status',
 'purpose',
 'initial_list_status']

In [26]:
for feature in categorical_features:
    temp=df.groupby(feature)['default_ind'].count()/len(df)
    temp_df=temp[temp>0.01].index
    df[feature]=np.where(df[feature].isin(temp_df),df[feature],'Rare_var')

In [27]:
#Label encoding for categorical features
from sklearn.preprocessing import LabelEncoder
for feature in categorical_features:
    labelencoder=LabelEncoder()
    df[feature]=labelencoder.fit_transform(df[feature])
df[categorical_features].head() 

,grade,sub_grade,home_ownership,verification_status,purpose,initial_list_status
0,0,0,0,0,1,1
1,4,21,2,0,2,1
2,0,3,0,0,1,1
3,3,18,0,0,2,1
4,0,2,0,0,1,1


In [28]:
#Feature scaling
scaling_feature=[feature for feature in df.columns if feature not in year_feature+['default_ind']]
len(scaling_feature)

31

In [29]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(df[scaling_feature])

MinMaxScaler()

In [30]:
df[scaling_feature]=scaler.transform(df[scaling_feature])

In [31]:
data=pd.concat([df['default_ind'].reset_index(drop=True),
                pd.DataFrame(df[scaling_feature],columns=scaling_feature)],axis=1)
data.head()

,default_ind,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,cred_hist_len_years
0,0,0.705882,0.0,0.000000,0.510655,0.000000,0.00,0.1,0.000000,0.741803,0.0,0.2,0.336591,0.0,0.0,0.000000,0.372090,0.0,0.663218,0.220207,0.084848,1.0,0.493347,0.616041,0.617998,0.431876,0.0,0.624296,0.000000,0.847510,0.665432,0.727273
1,0,0.500000,1.0,0.598226,0.311996,0.666667,0.84,0.7,0.666667,0.741803,0.0,0.4,0.334542,0.0,0.2,0.000000,0.604261,1.0,0.641759,0.210881,0.139394,1.0,0.360937,0.534333,0.496575,0.542405,0.0,0.554190,0.319914,0.685802,0.648262,0.636364
2,0,0.176471,0.0,0.091677,0.132548,0.000000,0.12,1.0,0.000000,0.683727,0.0,0.2,0.377773,1.0,0.2,0.617398,0.457565,0.0,0.621969,0.125389,0.181818,1.0,0.141778,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.713304,0.662213,0.727273
3,0,0.558824,0.0,0.534432,0.489374,0.500000,0.72,0.7,0.000000,0.753150,0.0,0.4,0.466180,0.0,0.0,0.000000,0.522295,0.0,0.738310,0.477202,0.157576,1.0,0.405081,0.000000,0.000000,0.000000,0.0,0.000000,0.469876,0.818705,0.686792,0.797980
4,0,0.411765,0.0,0.070553,0.305912,0.000000,0.08,0.2,0.000000,0.712050,0.0,0.2,0.394989,0.0,0.2,0.000000,0.403704,0.0,0.645760,0.223834,0.078788,1.0,0.303811,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.825247,0.648262,0.757576


In [32]:
data.to_csv('C:/Users/hp/Desktop/DSP/New folder (2)/Python/Credit risk/Preprocessed_Test_data.csv',index=False)